In [1]:
#! /usr/bin/env python3.6
import argparse
import argcomplete
from argcomplete.completers import ChoicesCompleter
from argcomplete.completers import EnvironCompleter
import requests
import datetime
import bthread
from bs4 import BeautifulSoup
import time
import pandas as pd
import re
import math

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import numpy as np
from IPython.display import clear_output

import sys
sys.path.append("../")
#import test.booking as book
import warnings
warnings.filterwarnings('ignore')


new

In [2]:
country = 'Sicily'
rooms = 1
adults = 2
checkin = '2023-09-01'
checkout = '2023-09-08'

In [3]:
## opciones selenium
opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
#para ocultarme como robot
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized') #empezar maximizado
opciones.add_argument('user.data-dir=selenium') #guarda las cookies
opciones.add_argument('--incognito')#incognito window

In [4]:
url_original = str(
    "https://www.booking.com/searchresults.en-gb.html?"
    "aid=304142&label"
    "=gen173nr-1DCAEoggJCAlhYSDNYBGiTAYgBAZgBLsIBCnd"
    "pbmRvd3MgMTDIAQzYAQPoAQGSAgF5qAID&"
    "sid=716ea5d78c4043fd78b7a1410d639e3d&checkin_month="
    "{checkin_m}&checkin_monthday={checkin_d}&checkin_year={checkin_y}&checkout_month={checkout_m}&"
    "checkout_monthday={checkout_d}&checkout_year={checkout_y}"
    "&class_interval=1&dest_id=125&dest_type=country&dtdisc=0&from_sf"
    "=1&genius_rate=1&no_rooms={rooms}&group_adults=2&group_children=0&inac=0&"
    "index_postcard=0&label_click=undef"
    "&no_rooms=1&postcard=0&raw_dest_type=country&room1="
    "A%2CA&sb_price"
    "_type=total&src=searchresults&src_elem=sb&ss={country}&ss_all="
    "0&ssb=empty&sshis=0&ssne={country}"
    "&ssne_untouched={country}&rows=15&offset=".format(
        rooms=rooms,
        country=country.replace(" ", "+"),
        checkin_m=checkin[1],
        checkin_d=checkin[2],
        checkin_y=checkin[0],
        checkout_m=checkout[1],
        checkout_d=checkout[2],
        checkout_y=checkout[0],
    )
)

In [5]:
url_original = str(
    'https://www.booking.com/searchresults.en-gb.html?'
    'ss=Sicily%2C+Italy&ssne=Sicily%2C+Italy&ssne_untouched=Sicily%2C+Italy'
    '&efdco=1&label'
    '=gen173nr-1DCAEoggJCAlhYSDNYBGiTAYgBAZgBLsIBCnd'
    'pbmRvd3MgMTDIAQzYAQPoAQGSAgF5qAID&'
    'sid=716ea5d78c4043fd78b7a1410d639e3d&'
    'aid=304142&'
    'lang=en-gb'
    '&sb=1&'
    'src_elem=sb&'
    'src=index&'
    'dest_id=909&'
    'dest_type=region&'
    'ac_position=0&'
    'ac_click_type=b&'
    'ac_langcode=en&'
    'ac_suggestion_list_length=5&'
    'search_selected=true&'
    'search_pageview_id=36746f79352b0064&'
    'ac_meta=GhAzNjc0NmY3OTM1MmIwMDY0IAAoATICZW46BFNpY2lAAEoAUAA%3D&'
    'checkin={checkin}&'
    'checkout={checkout}&'
    'group_adults={adults}&'
    'no_rooms={rooms}&'
    'group_children=0&'
    'sb_travel_purpose=leisure'.format(
        rooms=rooms,
        country=country.replace(" ", "+"),
        checkin=checkin,
        checkout=checkout,
        adults=adults,
    )
)

In [6]:
almacen = {
    'url':[], 
    'pagina':[]
}

In [7]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url_original)
driver.maximize_window()
driver.set_window_size(1920, 1080)
driver.implicitly_wait(30)
driver.find_element('css selector', '#onetrust-accept-btn-handler').click()
res = requests.get(url_original)
soup_numero = BeautifulSoup(res.content, 'html.parser')
pagina_numero = soup_numero.text
number = math.ceil(int(re.findall(r': (.*) properties', pagina_numero)[0].replace(',', '')) / 25)
try:
    for page in range (1,number):
        try:
            for i in range(1,50):
                print('item:', i, 'page:', page)
                try:
                    driver.find_element('xpath', '//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[{i}]/div[1]/div[2]/div/div[1]/div[1]/div/div[1]/div[1]/h3/a'.format(i=i)).click()
                    driver.implicitly_wait(15)
                    url = driver.current_url
                    almacen['url'].append(url)
                    res = requests.get(url)
                    soup = BeautifulSoup(res.content, 'html.parser')
                    pagina = soup.text
                    almacen['pagina'].append(pagina)
                    print('dict:', len(almacen['url']), len(almacen['pagina']))
                    driver.switch_to.window(driver.window_handles[1])
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    clear_output(wait=True)
                except:
                    pass
                clear_output
            driver.find_element('xpath', '//*[@id="search_results_table"]/div[2]/div/div/div/div[4]/div[2]/nav/div/div[3]/button').click()
            driver.implicitly_wait(2)
            clear_output(wait=True)
        except:
            pass
except:
    print('me rompí')
    #//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[5]/div[1]/div[2]/div/div[1]/div[1]/div/div[1]/div/h3/a
    #//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[3]/div[1]/div[2]/div/div[1]/div[1]/div/div[1]/div[1]/h3/a
    #search_results_table > div:nth-child(2) > div > div > div > div.d4924c9e74 > div:nth-child(4) > div.d20f4628d0 > div.b978843432 > div > div.a1b3f50dcd.b2fe1a41c3.a7c67ebfe5.d19ba76520.d14b211b4f > div.b1e6dd8416.aacd9d0b0a > div > div:nth-child(1) > div.dd023375f5 > h3 > a
    #search_results_table > div:nth-child(2) > div > div > div > div.d4924c9e74 > div:nth-child(6) > div.d20f4628d0 > div.b978843432 > div > div.a1b3f50dcd.b2fe1a41c3.a7c67ebfe5.d19ba76520.d14b211b4f > div.b1e6dd8416.aacd9d0b0a > div > div:nth-child(1) > div > h3 > a
#search_results_table > div:nth-chil
# Aur
# d(2) > div > div > div > div.d4924c9e74 > div:nth-child(4) > div.d20f4628d0 > div.b978843432 > div > div.a1b3f50dcd.b2fe1a41c3.a7c67ebfe5.d19ba76520.d14b211b4f > div.b1e6dd8416.aacd9d0b0a > div > div:nth-child(1) > div > h3 > a > div.fcab3ed991.a23c043802
##search_results_table > div:nth-child(2) > div > div > div > div.d4924c9e74 > div:nth-child(6) > div.d20f4628d0 > div.b978843432 > div > div.a1b3f50dcd.b2fe1a41c3.a7c67ebfe5.d19ba76520.d14b211b4f > div.b1e6dd8416.aacd9d0b0a > div > div:nth-child(1) > div > h3 > a > div.fcab3ed991.a23c043802
##search_results_table > div:nth-child(2) > div > div > div > div.d4924c9e74 > div:nth-child(8) > div.d20f4628d0 > div.b978843432 > div > div.a1b3f50dcd.b2fe1a41c3.a7c67ebfe5.d19ba76520.d14b211b4f > div.b1e6dd8416.aacd9d0b0a > div > div:nth-child(1) > div > h3 > a > div.fcab3ed991.a23c043802
#except:
#    driver.find_element('xpath', '//*[@id="__bui-c2718823-1"]').send_keys(city, Keys.TAB)
    

item: 35 page: 2


In [ ]:
len(almacen['url'])

In [ ]:
almacen['url']

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url_original)
driver.maximize_window()
driver.set_window_size(1920, 1080)
driver.implicitly_wait(30)
driver.find_element('css selector', '#onetrust-accept-btn-handler').click()
try:
    #driver.find_elements_by_xpath('')
    driver.find_element('xpath', '//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[3]/div[1]/div[2]/div/div[1]/div[1]/div/div[1]/div[1]/h3/a').click().click()
    driver.implicitly_wait(15)
    #almacen['url'].append(driver.getCurrentUrl())
    time.sleep(5)
    for handle in driver.window_handles:
        print('handle')
        driver.switch_to.window(handle)
    #p = driver.window_handles[0]
    #obtain browser tab window
    #c = driver.window_handles[1]
    #switch to tab browser
    #driver.switch_to.window(c)
    try:
        chwd = driver.window_handles
        print(chwd[0])
        for w in chwd:
        #switch focus to child window
            if(w!=p):
                driver.switch_to.window(w)
                page_source = driver.page_source
                time.sleep(1)
            url = driver.current_url
            almacen['url'].append(url)
            res = requests.get(url)
            soup = BeautifulSoup(res.content, 'html.parser')
            pagina = soup.text
            almacen['pagina'].append(pagina)
            #driver.current_url
            #print("The current url is:"+str(driver.current_url))
    except:
        pass
except:
    print('aloha')
    #//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[5]/div[1]/div[2]/div/div[1]/div[1]/div/div[1]/div/h3/a
    #//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[3]/div[1]/div[2]/div/div[1]/div[1]/div/div[1]/div[1]/h3/a
    #search_results_table > div:nth-child(2) > div > div > div > div.d4924c9e74 > div:nth-child(4) > div.d20f4628d0 > div.b978843432 > div > div.a1b3f50dcd.b2fe1a41c3.a7c67ebfe5.d19ba76520.d14b211b4f > div.b1e6dd8416.aacd9d0b0a > div > div:nth-child(1) > div.dd023375f5 > h3 > a
    #search_results_table > div:nth-child(2) > div > div > div > div.d4924c9e74 > div:nth-child(6) > div.d20f4628d0 > div.b978843432 > div > div.a1b3f50dcd.b2fe1a41c3.a7c67ebfe5.d19ba76520.d14b211b4f > div.b1e6dd8416.aacd9d0b0a > div > div:nth-child(1) > div > h3 > a
#search_results_table > div:nth-chil
# Aur
# d(2) > div > div > div > div.d4924c9e74 > div:nth-child(4) > div.d20f4628d0 > div.b978843432 > div > div.a1b3f50dcd.b2fe1a41c3.a7c67ebfe5.d19ba76520.d14b211b4f > div.b1e6dd8416.aacd9d0b0a > div > div:nth-child(1) > div > h3 > a > div.fcab3ed991.a23c043802
##search_results_table > div:nth-child(2) > div > div > div > div.d4924c9e74 > div:nth-child(6) > div.d20f4628d0 > div.b978843432 > div > div.a1b3f50dcd.b2fe1a41c3.a7c67ebfe5.d19ba76520.d14b211b4f > div.b1e6dd8416.aacd9d0b0a > div > div:nth-child(1) > div > h3 > a > div.fcab3ed991.a23c043802
##search_results_table > div:nth-child(2) > div > div > div > div.d4924c9e74 > div:nth-child(8) > div.d20f4628d0 > div.b978843432 > div > div.a1b3f50dcd.b2fe1a41c3.a7c67ebfe5.d19ba76520.d14b211b4f > div.b1e6dd8416.aacd9d0b0a > div > div:nth-child(1) > div > h3 > a > div.fcab3ed991.a23c043802
#except:
#    driver.find_element('xpath', '//*[@id="__bui-c2718823-1"]').send_keys(city, Keys.TAB)
    

In [ ]:
driver.window_handles[0]

In [ ]:
almacen['url']

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.booking.com/')
driver.maximize_window()
driver.set_window_size(1920, 1080)
driver.implicitly_wait(30)
driver.find_element('css selector', '#onetrust-accept-btn-handler').click()
try:
    driver.find_element('xpath', '//*[@id="ss"]').send_keys(city, Keys.TAB)
except:
    driver.find_element('xpath', '//*[@id="__bui-c2718823-1"]').send_keys(city, Keys.TAB)
    

In [ ]:
link = str('https://www.booking.com/searchresults.es.html?'
    'label'
    '=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYAQq4ARfIAQzYAQHoAQH4AQuIAgGoAgO4AszOv54GwAIB0gIkZjVlYTNmNmQtZGVhOS00MzY5LWFjZDAtZGY4NGVmYmRhNzFm2AIG4AIB&'
    'sid=a4916b536ec3ecd3b62ccc75112c9105&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.es.html%3Flabel%3Dgen173nr-1FCAEoggI46AdIM1gEaEaIAQGYAQq4ARfIAQzYAQHoAQH4AQuIAgGoAgO4AszOv54GwAIB0gIkZjVlYTNmNmQtZGVhOS00MzY5LWFjZDAtZGY4NGVmYmRhNzFm2AIG4AIB%26'
    'sid%3Da4916b536ec3ecd3b62ccc75112c9105%26sb_price'
    '_type%3Dtotal%26%26&ss={country}&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1')

old

In [ ]:
import requests
from scrapy.selector import Selector

# Prepare url
city     = 'Sicily'
main_url = 'https://www.airbnb.com'
city_url = f'{main_url}/s/{city}/homes/'

# Create selector
html = requests.get(city_url).content
sel  = Selector(text=html)

# Get hotels
hotels = sel.css('div._8ssblpx')
print('Number of hotels:', len(hotels))

# Find the next page url
next_page = sel.css('a._za9j7e ::attr(href)').extract_first()
print('Next page:', '\n', next_page)

In [ ]:
def get_booking_page(offset, rooms, country, startdate, enddate):
    """
    Make request to booking page and parse html
    :param offset:
    :return: html page
    """
    checkin: list = str(startdate).split("-")
    checkout: list = str(enddate).split("-")
    url: str = (
        "https://www.booking.com/searchresults.en-gb.html?"
        "aid=304142&label"
        "=gen173nr-1DCAEoggJCAlhYSDNYBGiTAYgBAZgBLsIBCnd"
        "pbmRvd3MgMTDIAQzYAQPoAQGSAgF5qAID&"
        "sid=716ea5d78c4043fd78b7a1410d639e3d&checkin_month="
        "{checkin_m}&checkin_monthday={checkin_d}&checkin_year={checkin_y}&checkout_month={checkout_m}&"
        "checkout_monthday={checkout_d}&checkout_year={checkout_y}"
        "&class_interval=1&dest_id=125&dest_type=country&dtdisc=0&from_sf"
        "=1&genius_rate=1&no_rooms={rooms}&group_adults=2&group_children=0&inac=0&"
        "index_postcard=0&label_click=undef"
        "&no_rooms=1&postcard=0&raw_dest_type=country&room1="
        "A%2CA&sb_price"
        "_type=total&src=searchresults&src_elem=sb&ss={country}&ss_all="
        "0&ssb=empty&sshis=0&ssne={country}"
        "&ssne_untouched={country}&rows=15&offset=".format(
            rooms=rooms,
            country=country.replace(" ", "+"),
            checkin_m=checkin[1],
            checkin_d=checkin[2],
            checkin_y=checkin[0],
            checkout_m=checkout[1],
            checkout_d=checkout[2],
            checkout_y=checkout[0],
        )
        + str(offset)
    )
    r = requests.get(
        url,
        headers={
            "User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0)"
            " Gecko/20100101 Firefox/48.0"
        },
    )
    html = r.content
    parsed_html = BeautifulSoup(html, "lxml")
    return parsed_html